In [33]:
import zipfile
import torch
import torch.nn as nn
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage import io, transform
import numpy as np
import pandas as pd
import matplotlib.pyplot as ply
import os
import imageio
% matplotlib inline

In [83]:
#https://www.kaggle.com/skainkaryam/basic-data-visualization-using-pytorch-dataset
class TGSSaltDataset(data.Dataset):
    
    def __init__(self, root_path, file_list):
        self.root_path = root_path
        self.file_list = file_list
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        if index not in range(0, len(self.file_list)):
            return self.__getitem__(np.random.randint(0, self.__len__()))
        
        file_id = self.file_list[index]
        
        image_folder = os.path.join(self.root_path, "images")
        image_path = os.path.join(image_folder, file_id + ".png")
        
        mask_folder = os.path.join(self.root_path, "masks")
        mask_path = os.path.join(mask_folder, file_id + ".png")
        
        image = np.array(imageio.imread(image_path), dtype=np.uint8)
        mask = np.array(imageio.imread(mask_path), dtype=np.uint8)
        
        return image, mask

In [84]:
df_depth = pd.read_csv('./data/train.csv')

In [85]:
train_path = "./data/train/"
train_file_list = list(df_depth['id'].values)

In [86]:
len(train_file_list)

4000

In [42]:
train_data = TGSSaltDataset(train_path, train_file_list)

In [61]:
train_path

'./data/train/'

In [ ]:
image_path

In [81]:
img = np.array(imageio.imread('./data/train/images/000e218f21.png'), dtype=np.uint8)

In [82]:
(img[:,:,0] == img[:,:,2]).min()

True

In [88]:
from PIL import Image
import glob


In [101]:
f = glob.glob('./data/train/images/*.png')[0]

In [106]:
f.split('\\')[-1].split('.')[0]

'000e218f21'

In [117]:
train_images = []
for filename in glob.glob('./data/train/images/*.png'): #assuming gif
    im=Image.open(filename)
    img_id = filename.split('\\')[-1].split('.')[0]
    train_images.append([img_id, np.array(imageio.imread(filename), dtype=np.uint8)])
df_train = pd.DataFrame(train_images, columns=['id', 'img'])

In [161]:
def load_img(img_path):
    images = []
    for filename in glob.glob(f'{img_path}/*.png'): #assuming gif
        img_id = filename.split('\\')[-1].split('.')[0]
        images.append([img_id, np.array(imageio.imread(filename), dtype=np.uint8)])
    return pd.DataFrame(images, columns=['img_id', 'img']).set_index('img_id')

In [162]:
df_train = load_img('./data/train/images')

In [164]:
df_train['mask'] = load_img('./data/train/masks/')['img']

In [171]:
df_train.iloc[1]['mask']

array([[  0,   0,   0, ..., 255, 255, 255],
       [  0,   0,   0, ..., 255, 255, 255],
       [  0,   0,   0, ..., 255, 255, 255],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)

In [172]:
df_test = load_img('./data/test/images')

In [174]:
np_train = np.r_[df_train.img.tolist()]

In [181]:
np_train.shape

(4000, 101, 101, 3)